<a href="https://colab.research.google.com/github/AsellaS2/gemma_chatbot/blob/main/Letter_Flex_GEMMA_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip install langchain
!pip install -U langchain-openai
!pip install langchain-community
!pip install datasets huggingface_hub
!pip install chromadb
!pip install pypdf

  Using cached chromadb-0.5.7-py3-none-any.whl.metadata (6.8 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached fastapi-0.115.0-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.30.6-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.6.6-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.19.2-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.48b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments
from langchain.llms import HuggingFacePipeline

In [ ]:
from huggingface_hub import login

#허깅페이스 액세스토큰
hf_token = str(input('허깅페이스 토큰을 입력하세요 : '))

# 허깅페이스 로그인
login(hf_token)

# 모델 불러오기 : Gemma-2b-it

In [ ]:
# 사전학습 모델 불러오기
BASE_MODEL = "google/gemma-2b-it" # ENG
# BASE_MODEL = 'beomi/gemma-ko-2b' # KOR

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map = "auto",torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)  #모델에 맞는 tokenizer 로드

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

토크나이저 내용 확인

In [ ]:
# 파이프라인 설정
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2048,  # 생성할 최대 토큰 수 설정
    #add_special_tokens=True,
    do_sample = True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    #torch_dtype=torch.bfloat16,
    #repetition_penalty = 1.2,
    #eos_token_id = tokenizer.eos_token_id,
    #pad_token_id = tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe) #HuggingFacePipeline으로 감싸기

The model 'GemmaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
<ipython-input-9-546a9e219e7c>:18: LangChainDeprecationWa

# 모델 학습

#RAG

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader   #pdf변환필요
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.output_parsers import StrOutputParser

# PDF 로드 및 문서 분할
loaders = [PyPDFLoader('/content/지원서.pdf')] #ENG
# loaders = [PyPDFLoader('/content/구글 머신러닝 지원서.pdf')] #KOR
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#문서 임베딩
model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2" # 문장 유사도 측정 성능이 높은 모델 사용. ENG
# model_name = 'BM-K/KoSimCSE-roberta-multitask' # KOR
model_kwargs = {"device": "cuda"} # CUDA 사용
encode_kwargs = {'normalize_embeddings': True} # vector 길이 정규화, 유사도 계산 용이
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs, # CPU 사용할 경우 사용 X
    encode_kwargs=encode_kwargs
)

<ipython-input-16-1606fb17a168>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 자식 문서 분할기 설정
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 50) # 텍스트를 chunk_size 분할함(300자).chunk_overlap size만큼 참조함.

# 벡터스토어 및 저장소 설정
vectorstore = Chroma(
    collection_name="full_documents", embedding_function= embedding
)
store = InMemoryStore() # 문서의 원본 텍스트를 저장하는 인메모리 저장소 객체입니다.

# 문서 검색 기능
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

<ipython-input-17-1bdc5c3a0b83>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [ ]:
retriever.add_documents(docs, ids=None)
topic = "Project experience"
sub_docs = vectorstore.similarity_search(topic)

In [ ]:
# 추출한 텍스트 출력
print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

글 길이: 225


an AI developer, gain project experience, and reinforce my sense of purpose by 
completing this program.  
[Enhancing AI Developer Skills]  
Through Coursera, I intend to learn effective project management techniques for deep


[Document(metadata={'doc_id': '1f071317-618e-4a77-ae99-1a5a31bf2a57', 'page': 2, 'source': '/content/지원서.pdf'}, page_content='an AI developer, gain project experience, and reinforce my sense of purpose by \ncompleting this program.  \n[Enhancing AI Developer Skills]  \nThrough Coursera, I intend to learn effective project management techniques for deep'), Document(metadata={'doc_id': '1f071317-618e-4a77-ae99-1a5a31bf2a57', 'page': 2, 'source': '/content/지원서.pdf'}, page_content='expertise in image classification and time series analysis.  \n[Project Experience]  \nI hope to participate in the Kaggle TPS competition during the boot camp. From past \ncompetition experiences, I have realized the significance of applying theory to practice.'), Document(metadata={'doc_id': '0e80a374-2b4d-4b47-9cad-5523dbcade92', 'page': 1, 'source': '/content/지원서.pdf'}, page_content='behavior and provide personalized product recommendations, making machine learning \nmodels a key component of a c

In [ ]:
retrieved_docs = retriever.get_relevant_documents(topic)

<ipython-input-20-ad15157d0af6>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(topic)


In [ ]:
print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 2408


courses on platforms like Fast Campus, implementing these theories in Python. During 
vacations, I furthered my efforts by earning relevant certifications.  
Currently, I am serving as a session leader in a deep learning society, where I implement 
machine learning and deep learning theories and share my work with team members via 
Slack. The curriculum of this boot camp convinced me that it offers the right 
environment for me to grow. As I approach graduation, I aim to strengthen my skills as 
an AI developer, gain project experience, and reinforce my sense of purpose by 
completing this program.  
[Enhancing AI Developer Skills]  
Through Coursera, I intend to learn effective project management techniques for deep 
learning. Completing a project requires knowledge beyond deep learning theory, 
including computer vision, hyperparameter tuning, and error validation. Coursera’s 
structure d courses will help me gain these skills. I also plan to explore deepfake 
technology

#PromptEngineering - 프롬포트 생성 (첫번째가 성능이 더 좋습니다.)

In [ ]:
# 프롬프트 템플릿 설정
text = retrieved_docs[0].page_content
topic = "Project experience"
prompt_template = """
Below is a self-introduction text written on the topic of {topic}.

{text}

Please revise this to fit the context in paragraph form.
"""

prompt = PromptTemplate(
    input_variables=["topic", "text"],
    template=prompt_template,
)



# LLM 체인 생성
llm_chain = prompt | llm

# LLM에 텍스트 전달 및 결과 출력
output = llm_chain.invoke({"topic": topic, "text": text})

In [ ]:
# 프롬프트 템플릿 설정

text = retrieved_docs[0].page_content

prompt_template = """

Proofread the following self-introduction statement themed around {topic}, and revise it to ensure it reads naturally and aligns well with the theme.\n

ConText : {text}\n

Please present the final version in the following format:

Topic: {topic}\n
Context:

"""

prompt = PromptTemplate(
    input_variables=["topic", "text"],
    template=prompt_template,
)

# LLM 체인 생성
llm_chain = prompt | llm
# LLM에 텍스트 전달 및 결과 출력
output = llm_chain.invoke({"topic": topic, "text": text})

# 결과 확인

In [ ]:
import textwrap

# 특정 키워드로 원하는 부분을 추출
start = output.find("here is the revised text:")
selected_text = output[start:]

wrapped_text = textwrap.fill(textwrap.dedent(selected_text), width=100)
print(wrapped_text)

.


나름 만족스러운 결과가 나온것같네요 ..?

In [ ]:
print(output)



Proofread the following self-introduction statement themed around Project experience, and revise it to ensure it reads naturally and aligns well with the theme.


ConText : courses on platforms like Fast Campus, implementing these theories in Python. During 
vacations, I furthered my efforts by earning relevant certifications.  
Currently, I am serving as a session leader in a deep learning society, where I implement 
machine learning and deep learning theories and share my work with team members via 
Slack. The curriculum of this boot camp convinced me that it offers the right 
environment for me to grow. As I approach graduation, I aim to strengthen my skills as 
an AI developer, gain project experience, and reinforce my sense of purpose by 
completing this program.  
[Enhancing AI Developer Skills]  
Through Coursera, I intend to learn effective project management techniques for deep 
learning. Completing a project requires knowledge beyond deep learning theory, 
including compute